In [ ]:
import shutil
import subprocess
import glob
from tqdm import tqdm
import numpy as np
import cv2
import os
from keras.applications.vgg16 import VGG16
from keras.models import Model 
#import fire
#from elapsedtimer import ElapsedTimer
import cv2
from keras.applications.vgg16 import preprocess_input


In [16]:
from keras import backend as K
import tensorflow as tf

NUM_PARALLEL_EXEC_UNITS = 16

config = tf.ConfigProto(intra_op_parallelism_threads = 16, 
         inter_op_parallelism_threads = 16, 
         allow_soft_placement = True, 
         device_count = {'CPU': 16 })

session = tf.Session(config=config)

K.set_session(session)

import os

os.environ["OMP_NUM_THREADS"] = str(16)
os.environ["KMP_BLOCKTIME"] = "30"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

In [17]:
video_dest = 'C:\\Project\\YouTubeClips'
feat_dir = 'C:\\Project\\features'
temp_dest = 'C:\\Project\\TempImages\\'
img_dim = 224
channels=3
batch_size=128
batch_cnn =128
frames_step=80

os.chdir('C:/Project')

In [18]:
def video_to_frames(video):
    os.chdir('C:/Project/YouTubeClips')
    if not os.path.isdir(temp_dest):
        os.mkdir(temp_dest)
    print (video)
    print(temp_dest)
    vidcap = cv2.VideoCapture(video)
    #os.chdir('C:/Project/TempImages')
    print (vidcap)
    def getFrame(sec):

        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read() 
        if hasFrames:
            cv2.imwrite(temp_dest+"image"+str(count)+".jpg", image)     # save frame as JPG file            
        return hasFrames
    sec = 0
    frameRate = 0.5 #//it will capture image in each 0.5 second
    count=1
    success = getFrame(sec)
    while success:
        count = count + 1
        
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

In [20]:
def video_to_frames_new(video):
    os.chdir('C:/Project/YouTubeClips')
    with open(os.devnull, "w") as ffmpeg_log:
        if os.path.exists(temp_dest):
            print(" cleanup: " + temp_dest + "/")
            print(video)
            shutil.rmtree(temp_dest)
        os.makedirs(temp_dest)
        video_to_frames_cmd = ["ffmpeg",                                       
                                   '-y',
                                   '-i', video,  
                                   '-vf', "fps=1", 
                                   '-qscale:v', "2", 
                                   '{0}/%04d.jpg'.format(temp_dest)]
        subprocess.call(video_to_frames_cmd,
                        stdout=ffmpeg_log, stderr=ffmpeg_log, shell=True)

In [21]:
def model_cnn_load():
         model = VGG16(weights = "imagenet", include_top=True,input_shape = 
                                  (img_dim,img_dim,channels))
         out = model.layers[-2].output
         model_final = Model(input=model.input,output=out)
         return model_final

In [22]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import Model 
import numpy as np
def model_rseNet_load():
     model = ResNet50(weights='imagenet')
     out = model.layers[-2].output
     model_final = Model(input=model.input,output=out)
     return model_final

In [23]:
def load_image(path):
        img = cv2.imread(path)
        img = cv2.resize(img,(img_dim,img_dim))
        return img 

In [24]:
#Loading VGG16 model
#model = model_cnn_load()
model = model_rseNet_load()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  if __name__ == '__main__':


In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

activation_108 (Activation)     (None, 56, 56, 256)  0           add_35[0][0]                     
__________________________________________________________________________________________________
res3a_branch2a (Conv2D)         (None, 28, 28, 128)  32896       activation_108[0][0]             
__________________________________________________________________________________________________
bn3a_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_109 (Activation)     (None, 28, 28, 128)  0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_109[0][0]             
__________________________________________________________________________________________________
bn3a_branc

__________________________________________________________________________________________________
bn3d_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3d_branch2c[0][0]             
__________________________________________________________________________________________________
add_39 (Add)                    (None, 28, 28, 512)  0           bn3d_branch2c[0][0]              
                                                                 activation_117[0][0]             
__________________________________________________________________________________________________
activation_120 (Activation)     (None, 28, 28, 512)  0           add_39[0][0]                     
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 14, 14, 256)  131328      activation_120[0][0]             
__________________________________________________________________________________________________
bn4a_branc

__________________________________________________________________________________________________
bn4d_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_131 (Activation)     (None, 14, 14, 256)  0           bn4d_branch2b[0][0]              
__________________________________________________________________________________________________
res4d_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_131[0][0]             
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_43 (Add)                    (None, 14, 14, 1024) 0           bn4d_branch2c[0][0]              
          

__________________________________________________________________________________________________
bn5b_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_142 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2a[0][0]              
__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_142[0][0]             
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_143 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2b[0][0]              
__________

In [26]:
def extract_feats_pretrained_cnn():
        
    model = model_rseNet_load()
    print('Model loaded') 
    if not os.path.isdir(feat_dir):
        os.mkdir(feat_dir)
        #print("save video feats to %s" % (dir_feat))
    video_list = glob.glob(os.path.join(video_dest, '*.avi'))
        #print video_list 
        
    for i in range (0,len(video_list)-1):
        video = video_list[i]
        video_id = video.split("YouTubeClips")[-1]
        video_id = video_id[1:]    
        outfile = os.path.join(feat_dir, video_id + '.npy')
        if not os.path.isfile(outfile):
            #print (video)    
            print(f'Processing video {video_id}')
            os.chdir('C:/Project')
            video_to_frames(video_id)
            os.chdir('C:/Project')
            image_list = sorted(glob.glob(os.path.join(temp_dest, '*.jpg')))
            print (image_list)
            samples = np.round(np.linspace(0, len(image_list) - 1,frames_step))
            #print (samples)
            image_list = [image_list[int(sample)] for sample in samples]
            images = np.zeros((len(image_list),img_dim,img_dim,channels))
            for i in range(len(image_list)):
                img = load_image(image_list[i])
                #img = preprocess_input(img)
                images[i] = img
            images = np.array(images)
            print('preparing start')
            for i in range(len(images)):
                images[i] = preprocess_input(images[i])
                #img = preprocess_input(img)    
            print('preparing finished')
            fc_feats = model.predict(images,batch_size=batch_cnn)
            img_feats = np.array(fc_feats)
            outfile = os.path.join(feat_dir, video_id + '.npy')
            np.save(outfile, img_feats)
            # cleanup
            shutil.rmtree(temp_dest)

In [ ]:
extract_feats_pretrained_cnn()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  if __name__ == '__main__':


Model loaded
Processing video -4wsuPCjDBc_5_15.avi
-4wsuPCjDBc_5_15.avi
C:\Project\TempImages\
<VideoCapture 00000241A677E6B0>
['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image14.jpg', 'C:\\Project\\TempImages\\image15.jpg', 'C:\\Project\\TempImages\\image16.jpg', 'C:\\Project\\TempImages\\image17.jpg', 'C:\\Project\\TempImages\\image18.jpg', 'C:\\Project\\TempImages\\image19.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image20.jpg', 'C:\\Project\\TempImages\\image21.jpg', 'C:\\Project\\TempImages\\image22.jpg', 'C:\\Project\\TempImages\\image23.jpg', 'C:\\Project\\TempImages\\image24.jpg', 'C:\\Project\\TempImages\\image25.jpg', 'C:\\Project\\TempImages\\image26.jpg', 'C:\\Project\\TempImages\\image27.jpg', 'C:\\Project\\TempImages\\image28.jpg', 'C:\\Project\\TempImages\\image29.

['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\image4.jpg', 'C:\\Project\\TempImages\\image5.jpg', 'C:\\Project\\TempImages\\image6.jpg', 'C:\\Project\\TempImages\\image7.jpg', 'C:\\Project\\TempImages\\image8.jpg', 'C:\\Project\\TempImages\\image9.jpg']
preparing start
preparing finished
Processing video -FugkxLmGO4_5_16.avi
-FugkxLmGO4_5_16.avi
C:\Project\TempImages\
<VideoCapture 00000241A677E6B0>
['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image14.jpg', 'C:\\Project\\TempImages\\image15.jpg', 'C:\\Project\\TempImages\\image16.jpg', 'C:\\Project\\TempI

C:\Project\TempImages\
<VideoCapture 00000241A677E6B0>
['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image14.jpg', 'C:\\Project\\TempImages\\image15.jpg', 'C:\\Project\\TempImages\\image16.jpg', 'C:\\Project\\TempImages\\image17.jpg', 'C:\\Project\\TempImages\\image18.jpg', 'C:\\Project\\TempImages\\image19.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image20.jpg', 'C:\\Project\\TempImages\\image21.jpg', 'C:\\Project\\TempImages\\image22.jpg', 'C:\\Project\\TempImages\\image23.jpg', 'C:\\Project\\TempImages\\image24.jpg', 'C:\\Project\\TempImages\\image25.jpg', 'C:\\Project\\TempImages\\image26.jpg', 'C:\\Project\\TempImages\\image27.jpg', 'C:\\Project\\TempImages\\image28.jpg', 'C:\\Project\\TempImages\\image29.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\i

['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image14.jpg', 'C:\\Project\\TempImages\\image15.jpg', 'C:\\Project\\TempImages\\image16.jpg', 'C:\\Project\\TempImages\\image17.jpg', 'C:\\Project\\TempImages\\image18.jpg', 'C:\\Project\\TempImages\\image19.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image20.jpg', 'C:\\Project\\TempImages\\image21.jpg', 'C:\\Project\\TempImages\\image22.jpg', 'C:\\Project\\TempImages\\image23.jpg', 'C:\\Project\\TempImages\\image24.jpg', 'C:\\Project\\TempImages\\image25.jpg', 'C:\\Project\\TempImages\\image26.jpg', 'C:\\Project\\TempImages\\image27.jpg', 'C:\\Project\\TempImages\\image28.jpg', 'C:\\Project\\TempImages\\image29.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\image30.jpg', 'C:\\Project\\TempImages\\image31.jpg', 'C

...

['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\image4.jpg', 'C:\\Project\\TempImages\\image5.jpg', 'C:\\Project\\TempImages\\image6.jpg', 'C:\\Project\\TempImages\\image7.jpg', 'C:\\Project\\TempImages\\image8.jpg', 'C:\\Project\\TempImages\\image9.jpg']
preparing start
preparing finished
Processing video WPG-BIWOrG4_635_640.avi
WPG-BIWOrG4_635_640.avi
C:\Project\TempImages\
<VideoCapture 000002414B261490>
['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\image4.jpg', 'C:\\Project\\TempImages\\image5.jpg', 'C:\\Project\\TempImages\\image6.jpg', 'C:\\Project\\TempImages\\image7.jpg', 'C:\\Project\\TempImages\\image8.jpg', 'C:\\Project\\TempImages\\image9.jpg']
preparing start
preparing finished
Processing video WPG-BIWOr

['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image14.jpg', 'C:\\Project\\TempImages\\image15.jpg', 'C:\\Project\\TempImages\\image16.jpg', 'C:\\Project\\TempImages\\image17.jpg', 'C:\\Project\\TempImages\\image18.jpg', 'C:\\Project\\TempImages\\image19.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image20.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\image4.jpg', 'C:\\Project\\TempImages\\image5.jpg', 'C:\\Project\\TempImages\\image6.jpg', 'C:\\Project\\TempImages\\image7.jpg', 'C:\\Project\\TempImages\\image8.jpg', 'C:\\Project\\TempImages\\image9.jpg']
preparing start
preparing finished
Processing video WTf5EgVY5uU_100_104.avi
WTf5EgVY5uU_100_104.avi
C:\Project\TempImages\
<VideoCapture 000002414B2613B0>
['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\

['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image14.jpg', 'C:\\Project\\TempImages\\image15.jpg', 'C:\\Project\\TempImages\\image16.jpg', 'C:\\Project\\TempImages\\image17.jpg', 'C:\\Project\\TempImages\\image18.jpg', 'C:\\Project\\TempImages\\image19.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image20.jpg', 'C:\\Project\\TempImages\\image21.jpg', 'C:\\Project\\TempImages\\image22.jpg', 'C:\\Project\\TempImages\\image23.jpg', 'C:\\Project\\TempImages\\image24.jpg', 'C:\\Project\\TempImages\\image25.jpg', 'C:\\Project\\TempImages\\image26.jpg', 'C:\\Project\\TempImages\\image27.jpg', 'C:\\Project\\TempImages\\image28.jpg', 'C:\\Project\\TempImages\\image29.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\image30.jpg', 'C:\\Project\\TempImages\\image31.jpg', 'C

['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\image4.jpg', 'C:\\Project\\TempImages\\image5.jpg', 'C:\\Project\\TempImages\\image6.jpg', 'C:\\Project\\TempImages\\image7.jpg', 'C:\\Project\\TempImages\\image8.jpg', 'C:\\Project\\TempImages\\image9.jpg']
preparing start
preparing finished
Processing video WTx-K045yQM_85_98.avi
WTx-K045yQM_85_98.avi
C:\Project\TempImages\
<VideoCapture 000002414B261AB0>
['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image14.jpg', 'C:\\Project\\TempImages\\image15.jpg', 'C:\\Project\\TempImages\\image16.jpg', 'C:\\Project\\TempImages\\image17.jpg', 'C:\\Project\\Tem

['C:\\Project\\TempImages\\image1.jpg', 'C:\\Project\\TempImages\\image10.jpg', 'C:\\Project\\TempImages\\image11.jpg', 'C:\\Project\\TempImages\\image12.jpg', 'C:\\Project\\TempImages\\image13.jpg', 'C:\\Project\\TempImages\\image14.jpg', 'C:\\Project\\TempImages\\image15.jpg', 'C:\\Project\\TempImages\\image16.jpg', 'C:\\Project\\TempImages\\image17.jpg', 'C:\\Project\\TempImages\\image18.jpg', 'C:\\Project\\TempImages\\image19.jpg', 'C:\\Project\\TempImages\\image2.jpg', 'C:\\Project\\TempImages\\image20.jpg', 'C:\\Project\\TempImages\\image21.jpg', 'C:\\Project\\TempImages\\image22.jpg', 'C:\\Project\\TempImages\\image23.jpg', 'C:\\Project\\TempImages\\image24.jpg', 'C:\\Project\\TempImages\\image25.jpg', 'C:\\Project\\TempImages\\image26.jpg', 'C:\\Project\\TempImages\\image27.jpg', 'C:\\Project\\TempImages\\image28.jpg', 'C:\\Project\\TempImages\\image29.jpg', 'C:\\Project\\TempImages\\image3.jpg', 'C:\\Project\\TempImages\\image30.jpg', 'C:\\Project\\TempImages\\image31.jpg', 'C